## The Project Codes to build a Recommender System for a Restaurant 

## Company in East Toronto Canada

In order to build the recommender system, we will begin by obtaining data about Toronto Postal codes from Wikipaedia.

The data is already available via this link:https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [1]:
!pip install bs4 #To install Beautifhul soup package

     |████████████████████████████████| 122kB 5.1MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
from bs4 import BeautifulSoup
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
import geopy.geocoders # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries are imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         394 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5.

Import the Toronto Locational data provided by the company

In [10]:
Toronto_data = pd.read_csv("Toronto_data.csv")
Toronto_data.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)
Toronto_data.head()

,Unnamed: 0,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Let's check how many neighborhoods that exist in the city

In [11]:
print('There are {} boruoghs and {} neighborhhods in Toronto Canda'.format(len(Toronto_data['Borough'].unique()),Toronto_data['Neighborhood'].shape[0]))

There are 10 boruoghs and 103 neighborhhods in Toronto Canda


We can see that the data already contains postal codes and neighborhoods in Toronto. 
Let's create a map of the city with it's postal codes.

In [5]:
#First we define the Latitides and Longitudes of Toronto
latitude = 43.6932; longitude = -79.3832
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Borough'], Toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

At this point, we will then focus on East Toronto

In [12]:
East_T_data = Toronto_data[Toronto_data['Borough']=='East Toronto'].reset_index(drop=True)
East_T_data.drop(columns='Unnamed: 0',inplace=True)
East_T_data = East_T_data.groupby('Neighborhood').mean().reset_index(drop=False)
East_T_data.head()

,Neighborhood,Latitude,Longitude
0,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
1,"India Bazaar, The Beaches West",43.668999,-79.315572
2,Studio District,43.659526,-79.340923
3,The Beaches,43.676357,-79.293031
4,"The Danforth West, Riverdale",43.679557,-79.352188


Take a look at what East Toronto looks like on map

In [15]:
borough = 'East Toronto, Toronto'
ET_latitude = 43.6912
ET_longitude = -79.3417
print('The geograpical coordinate of "East Toronto" are: {}, {}.'.format(ET_latitude, ET_longitude))

map_East_T = folium.Map(location=[ET_latitude, ET_longitude], zoom_start=11.5)

# add markers to map
for lat, lng, label in zip(East_T_data['Latitude'], East_T_data['Longitude'], East_T_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 10,
        popup = label,
        color ='blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7).add_to(map_East_T)  
    
map_East_T

The geograpical coordinate of "East Toronto" are: 43.6912, -79.3417.


Explore the neighborhoods of East Toronto with Foursquare API

Create a function to extract the venues for each postal code in East Toronto within a radius of 1000m from Foursquare

In [35]:
East_T_data['Neighborhood'].unique()

array(['The Beaches', 'The Danforth West, Riverdale',
       'India Bazaar, The Beaches West', 'Studio District',
       'Business reply mail Processing Centre, South Central Letter Processing Plant Toronto'],
      dtype=object)

In [16]:
#Begin from the Beaches
radius = 1000 
LIMIT = 500 
Beaches_latitude = East_T_data.loc[3, 'Latitude'] # neighborhood latitude value
Beaches_longitude = East_T_data.loc[3, 'Longitude'] # neighborhood longitude value
Beaches_name = East_T_data.loc[3, 'Neighborhood'] # neighbourhood

#Define the Foursquare url with access token 
CLIENT_ID = 'XULWLT1NWIPEEIIKBKDQ04OMI0BBOIRCPI30CKXQLCB414CE' # your Foursquare ID
CLIENT_SECRET = 'IYFPNDKEXKT1QTY0QGD2XVVLQWP4HBKGIXCKK0SWMK24TUCY' # your Foursquare Secret
ACCESS_TOKEN = 'XACUHXS1D2MDK40MYPDR1W2Q0KM0V5MKEEO5VYRY4UMN5KTO'
VERSION = '20180605' # Foursquare API version
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&oauth_token={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN, Beaches_latitude, Beaches_longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()

#Define a function to get the category of the row
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
Beaches_nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
Beaches_nearby_venues =Beaches_nearby_venues.loc[:, filtered_columns]

# filter the category for each row
Beaches_nearby_venues['venue.categories'] = Beaches_nearby_venues.apply(get_category_type, axis=1)
Beaches_nearby_venues['Neighborhood'] = East_T_data.loc[3,'Neighborhood']

print('Number of venues returned: {}'.format(Beaches_nearby_venues.shape[0]))
Beaches_nearby_venues.head(10)

Number of venues returned: 84


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,venue.name,venue.categories,venue.location.lat,venue.location.lng,Neighborhood
0,Glen Manor Ravine,Trail,43.676821,-79.293942,The Beaches
1,Tori's Bakeshop,Vegetarian / Vegan Restaurant,43.672114,-79.290331,The Beaches
2,Beaches Bake Shop,Bakery,43.680363,-79.289692,The Beaches
3,The Beech Tree,Gastropub,43.680493,-79.288846,The Beaches
4,Ed's Real Scoop,Ice Cream Shop,43.672630,-79.287993,The Beaches
5,The Fox Theatre,Indie Movie Theater,43.672801,-79.287272,The Beaches
6,Bagels On Fire,Bagel Shop,43.672864,-79.286784,The Beaches
7,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734,The Beaches
8,Mastermind Toys,Toy / Game Store,43.671453,-79.293971,The Beaches
9,Veloute Bistro,French Restaurant,43.672267,-79.289584,The Beaches


Now, we will get venues for The Danforth West, Riverdale

In [55]:
#Begin from the Danforth
radius = 1000 
LIMIT = 500 
Danforth_latitude = East_T_data.loc[1, 'Latitude'] # neighborhood latitude value
Danforth_longitude = East_T_data.loc[1, 'Longitude'] # neighborhood longitude value
Danforth_name = East_T_data.loc[1, 'Neighborhood'] # neighbourhood

#Define the Foursquare url with access token 
CLIENT_ID = 'XULWLT1NWIPEEIIKBKDQ04OMI0BBOIRCPI30CKXQLCB414CE' # your Foursquare ID
CLIENT_SECRET = 'IYFPNDKEXKT1QTY0QGD2XVVLQWP4HBKGIXCKK0SWMK24TUCY' # your Foursquare Secret
ACCESS_TOKEN = 'XACUHXS1D2MDK40MYPDR1W2Q0KM0V5MKEEO5VYRY4UMN5KTO'
VERSION = '20180605' # Foursquare API version
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&oauth_token={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN, Beaches_latitude, Beaches_longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()

#Define a function to get the category of the row
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
Danforth_nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
Danforth_nearby_venues =Danforth_nearby_venues.loc[:, filtered_columns]

# filter the category for each row
Danforth_nearby_venues['venue.categories'] = Danforth_nearby_venues.apply(get_category_type, axis=1)
Danforth_nearby_venues['Neighborhood'] = East_T_data.loc[1,'Neighborhood']
print('Number of venues returned: {}'.format(Beaches_nearby_venues.shape[0]))
Danforth_nearby_venues.head()

Number of venues returned: 84


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,venue.name,venue.categories,venue.location.lat,venue.location.lng,Neighborhood
0,Glen Manor Ravine,Trail,43.676821,-79.293942,"India Bazaar, The Beaches West"
1,Tori's Bakeshop,Vegetarian / Vegan Restaurant,43.672114,-79.290331,"India Bazaar, The Beaches West"
2,Beaches Bake Shop,Bakery,43.680363,-79.289692,"India Bazaar, The Beaches West"
3,The Beech Tree,Gastropub,43.680493,-79.288846,"India Bazaar, The Beaches West"
4,Ed's Real Scoop,Ice Cream Shop,43.672630,-79.287993,"India Bazaar, The Beaches West"


Now, we will get venues for India Bazaar, The Beaches West

In [56]:
#Then Bazaar
radius = 1000 
LIMIT = 500 
Bazaar_latitude = East_T_data.loc[0, 'Latitude'] # neighborhood latitude value
Bazaar_longitude = East_T_data.loc[0, 'Longitude'] # neighborhood longitude value
Barzaar_name = East_T_data.loc[0, 'Neighborhood'] # neighbourhood

#Define the Foursquare url with access token 
CLIENT_ID = 'XULWLT1NWIPEEIIKBKDQ04OMI0BBOIRCPI30CKXQLCB414CE' # your Foursquare ID
CLIENT_SECRET = 'IYFPNDKEXKT1QTY0QGD2XVVLQWP4HBKGIXCKK0SWMK24TUCY' # your Foursquare Secret
ACCESS_TOKEN = 'XACUHXS1D2MDK40MYPDR1W2Q0KM0V5MKEEO5VYRY4UMN5KTO'
VERSION = '20180605' # Foursquare API version
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&oauth_token={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN, Beaches_latitude, Beaches_longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()

#Define a function to get the category of the row
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
Bazaar_nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
Bazaar_nearby_venues =Bazaar_nearby_venues.loc[:, filtered_columns]

# filter the category for each row
Bazaar_nearby_venues['venue.categories'] = Bazaar_nearby_venues.apply(get_category_type, axis=1)
Bazaar_nearby_venues['Neighborhood'] = East_T_data.loc[0,'Neighborhood']
print('Number of venues returned: {}'.format(Beaches_nearby_venues.shape[0]))
Bazaar_nearby_venues.head()

Number of venues returned: 84


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,venue.name,venue.categories,venue.location.lat,venue.location.lng,Neighborhood
0,Glen Manor Ravine,Trail,43.676821,-79.293942,"Business reply mail Processing Centre, South C..."
1,Tori's Bakeshop,Vegetarian / Vegan Restaurant,43.672114,-79.290331,"Business reply mail Processing Centre, South C..."
2,Beaches Bake Shop,Bakery,43.680363,-79.289692,"Business reply mail Processing Centre, South C..."
3,The Beech Tree,Gastropub,43.680493,-79.288846,"Business reply mail Processing Centre, South C..."
4,Ed's Real Scoop,Ice Cream Shop,43.672630,-79.287993,"Business reply mail Processing Centre, South C..."


Now, we will go for venues in Studio District

In [58]:
radius = 1000 
LIMIT = 500 
Bazaar_latitude = East_T_data.loc[2, 'Latitude'] # neighborhood latitude value
Bazaar_longitude = East_T_data.loc[2, 'Longitude'] # neighborhood longitude value
Barzaar_name = East_T_data.loc[2, 'Neighborhood'] # neighbourhood

#Define the Foursquare url with access token 
CLIENT_ID = 'XULWLT1NWIPEEIIKBKDQ04OMI0BBOIRCPI30CKXQLCB414CE' # your Foursquare ID
CLIENT_SECRET = 'IYFPNDKEXKT1QTY0QGD2XVVLQWP4HBKGIXCKK0SWMK24TUCY' # your Foursquare Secret
ACCESS_TOKEN = 'XACUHXS1D2MDK40MYPDR1W2Q0KM0V5MKEEO5VYRY4UMN5KTO'
VERSION = '20180605' # Foursquare API version
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&oauth_token={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN, Beaches_latitude, Beaches_longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()

#Define a function to get the category of the row
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
Studio_nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
Studio_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
Studio_nearby_venues =Studio_nearby_venues.loc[:, filtered_columns]

# filter the category for each row
Studio_nearby_venues['venue.categories'] = Studio_nearby_venues.apply(get_category_type, axis=1)
Studio_nearby_venues['Neighborhood'] = East_T_data.loc[2,'Neighborhood']
print('Number of venues returned: {}'.format(Beaches_nearby_venues.shape[0]))
Studio_nearby_venues.head()

Number of venues returned: 84


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,venue.name,venue.categories,venue.location.lat,venue.location.lng,Neighborhood
0,Glen Manor Ravine,Trail,43.676821,-79.293942,Studio District
1,Tori's Bakeshop,Vegetarian / Vegan Restaurant,43.672114,-79.290331,Studio District
2,Beaches Bake Shop,Bakery,43.680363,-79.289692,Studio District
3,The Beech Tree,Gastropub,43.680493,-79.288846,Studio District
4,Ed's Real Scoop,Ice Cream Shop,43.672630,-79.287993,Studio District


We will finally get the data for venues  in Business reply mail Processing Centre, South Central Letter Processing Plant Toronto'

In [59]:
#Begin from the Beaches
radius = 1000 
LIMIT = 500 
South_Central_latitude = East_T_data.loc[0, 'Latitude'] # neighborhood latitude value
South_Central_longitude = East_T_data.loc[0, 'Longitude'] # neighborhood longitude value
South_Central_name = East_T_data.loc[0, 'Neighborhood'] # neighbourhood

#Define the Foursquare url with access token 
CLIENT_ID = 'XULWLT1NWIPEEIIKBKDQ04OMI0BBOIRCPI30CKXQLCB414CE' # your Foursquare ID
CLIENT_SECRET = 'IYFPNDKEXKT1QTY0QGD2XVVLQWP4HBKGIXCKK0SWMK24TUCY' # your Foursquare Secret
ACCESS_TOKEN = 'XACUHXS1D2MDK40MYPDR1W2Q0KM0V5MKEEO5VYRY4UMN5KTO'
VERSION = '20180605' # Foursquare API version
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&oauth_token={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN, Beaches_latitude, Beaches_longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()

#Define a function to get the category of the row
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
South_Central_nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
South_Central_nearby_venues =South_Central_nearby_venues.loc[:, filtered_columns]

# filter the category for each row
South_Central_nearby_venues['venue.categories'] = South_Central_nearby_venues.apply(get_category_type, axis=1)
South_Central_nearby_venues['Neighborhood'] = East_T_data.loc[0,'Neighborhood']
South_Central_nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,venue.name,venue.categories,venue.location.lat,venue.location.lng,Neighborhood
0,Glen Manor Ravine,Trail,43.676821,-79.293942,"Business reply mail Processing Centre, South C..."
1,Tori's Bakeshop,Vegetarian / Vegan Restaurant,43.672114,-79.290331,"Business reply mail Processing Centre, South C..."
2,Beaches Bake Shop,Bakery,43.680363,-79.289692,"Business reply mail Processing Centre, South C..."
3,The Beech Tree,Gastropub,43.680493,-79.288846,"Business reply mail Processing Centre, South C..."
4,Ed's Real Scoop,Ice Cream Shop,43.672630,-79.287993,"Business reply mail Processing Centre, South C..."


Join all five neighbourhoods data

In [60]:
All_venues = pd.concat([Beaches_nearby_venues,Danforth_nearby_venues,Bazaar_nearby_venues,Studio_nearby_venues,South_Central_nearby_venues], join='outer')
All_venues.head()


,venue.name,venue.categories,venue.location.lat,venue.location.lng,Neighborhood
0,Glen Manor Ravine,Trail,43.676821,-79.293942,The Beaches
1,Tori's Bakeshop,Vegetarian / Vegan Restaurant,43.672114,-79.290331,The Beaches
2,Beaches Bake Shop,Bakery,43.680363,-79.289692,The Beaches
3,The Beech Tree,Gastropub,43.680493,-79.288846,The Beaches
4,Ed's Real Scoop,Ice Cream Shop,43.672630,-79.287993,The Beaches


Hurray, we are happy to move on. Let's save the data.

In [61]:
All_venues.to_csv('All_East_Toronto_Venues.csv')

View how many venues we got

In [3]:
Venues_data = pd.read_csv('All_East_Toronto_Venues.csv')

In [4]:
print('We have got {} unique venues from the five Neighbourhoods in East Toronto'.format(Venues_data['venue.name'].unique().shape[0]))

We have got 82 unique venues from the five Neighbourhoods in East Toronto


In [5]:
Venues_data['Neighborhood'].unique()

array(['The Beaches', 'India Bazaar, The Beaches West',
       'Business reply mail Processing Centre, South Central Letter Processing Plant Toronto',
       'Studio District'], dtype=object)

Go on to build a clustering algorithm that will help to choose the best venue out of the neighborhoods venues.

In [5]:
#Trim the names of the columns to pick the last 
Venues_data.columns = [columns.split('.')[-1] for columns in Venues_data.columns]
#Drop the Unnamed:0 column
#Venues_data.drop('Unnamed: 0',axis=1,inplace=True)
Venues_data.tail()
#Venues_data.groupby('Neighborhood').count()

,Unnamed: 0,name,categories,lat,lng,Neighborhood
415,79,Honey Bee Restaurant,Chinese Restaurant,43.670409,-79.298406,"Business reply mail Processing Centre, South C..."
416,80,Dyson's valu-mart,Grocery Store,43.673271,-79.285854,"Business reply mail Processing Centre, South C..."
417,81,The Nail Curve,Cosmetics Shop,43.680801,-79.284602,"Business reply mail Processing Centre, South C..."
418,82,Gymboree,Kids Store,43.680600,-79.283779,"Business reply mail Processing Centre, South C..."
419,83,Pizzaville,Pizza Place,43.669704,-79.301244,"Business reply mail Processing Centre, South C..."


Using the category data, get the venues that are focused on selling food and snacks.

In [24]:
Venues_data['categories'].unique()

array(['Trail', 'Vegetarian / Vegan Restaurant', 'Bakery', 'Gastropub',
       'Ice Cream Shop', 'Indie Movie Theater', 'Bagel Shop',
       'Health Food Store', 'Toy / Game Store', 'French Restaurant',
       'Coffee Shop', 'Pub', 'Park', 'Beach', 'Breakfast Spot',
       'Mexican Restaurant', 'Nail Salon', 'Japanese Restaurant',
       'Juice Bar', 'Bar', 'Cupcake Shop', 'Mediterranean Restaurant',
       'Liquor Store', 'Tea Room', 'Diner', 'Indian Restaurant',
       'Grocery Store', 'Thai Restaurant', 'Pharmacy',
       'Caribbean Restaurant', 'Burger Joint', 'Sandwich Place',
       'Ramen Restaurant', 'Café', 'Chocolate Shop', 'Greek Restaurant',
       'Asian Restaurant', 'Bank', 'Pizza Place', 'Beer Store',
       'Business Service', 'Convenience Store', 'College Gym',
       'Shipping Store', 'Mattress Store', "Women's Store", 'Cheese Shop',
       'Pet Store', 'Nightlife Spot', 'Home Service', 'Restaurant',
       'Mobile Phone Shop', 'Chinese Restaurant', 'Cosmetics Shop',


In [6]:
Edibles_data = Venues_data.loc[(Venues_data['categories']=='Vegetarian / Vegan Restaurant') | (Venues_data['categories']== 'Bakery')|(Venues_data['categories']== 'Gastropub')| (Venues_data['categories']=='Bagel Shop')|(Venues_data['categories']=='Health Food Store')|(Venues_data['categories']=='French Restaurant')|
                     (Venues_data['categories']=='Breakfast Spot')|(Venues_data['categories']=='Mexican Restaurant')|(Venues_data['categories']=='Japanese Restaurant')|(Venues_data['categories']=='Diner')|(Venues_data['categories']=='Indian Restaurant')|(Venues_data['categories']=='Grocery Store')|(Venues_data['categories']=='Thai Restaurant')|(Venues_data['categories']=='Caribbean Restaurant')|
                     (Venues_data['categories']=='Ramen Restaurant')|(Venues_data['categories']=='Greek Restaurant')|(Venues_data['categories']=='Asian Restaurant')|(Venues_data['categories']=='Restaurant')|(Venues_data['categories']=='Chinese Restaurant')|(Venues_data['categories']=='Cupcake Shop')|(Venues_data['categories']=='Burger Joint')|(Venues_data['categories']=='Sandwich Place')|(Venues_data['categories']=='Cheese Shop')]

In [32]:
Edibles_data['Neighborhood'].value_counts()

Business reply mail Processing Centre, South Central Letter Processing Plant Toronto    66
India Bazaar, The Beaches West                                                          33
Studio District                                                                         33
The Beaches                                                                             33
Name: Neighborhood, dtype: int64

It is interesting to note that though we have seemingly 5 neighborhoods, only four are unique. 

This suggests there could be so much similarity between the neighborhoods

Next is to one-hot encode the data for clustering

In [7]:
Edibles_onehot = pd.get_dummies(Edibles_data, prefix="", prefix_sep="")
Edibles_onehot['Neighborhood'] = Edibles_data['Neighborhood'] #Add the neighbourhood column
fixed_columns = [Edibles_onehot.columns[-1]] + list(Edibles_onehot.columns[:-1])
Edibles_onehot = Edibles_onehot[fixed_columns]
Edibles_onehot.reset_index(drop=True,inplace=True)
Edibles_onehot.head()

,Neighborhood,Unnamed: 0,lat,lng,Bagels On Fire,Beacher Cafe,Beaches Bake Shop,Bikkuri Japanese Restaurant,COBS Bread,Cool Runnings,Delhi Bistro,Dyson's valu-mart,Foodland - Toronto,Garden Gate Resturant,Green Basil,Hero Certified Burgers,Honey Bee Restaurant,Honey's Beestro,Isabella’s Boutique Restaurant,Kyouka Ramen,Life Is Sweet,No Bull Burger,Souvlaki Hut,Subway,Sunset Grill,The Art of Cheese,The Beech Tree,The Big Carrot Natural Food Market,The Goof,The Real Jerk Beaches,The Salty Dog,Tori's Bakeshop,Veloute Bistro,Vi Vetha,Xola,Yumei Sushi,Asian Restaurant,Bagel Shop,Bakery,Breakfast Spot,Burger Joint,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Cupcake Shop,Diner,French Restaurant,Gastropub,Greek Restaurant,Grocery Store,Health Food Store,Indian Restaurant,Japanese Restaurant,Mexican Restaurant,Ramen Restaurant,Restaurant,Sandwich Place,Thai Restaurant,Vegetarian / Vegan Restaurant,"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto","India Bazaar, The Beaches West",Studio District,The Beaches
0,The Beaches,1,43.672114,-79.290331,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
1,The Beaches,2,43.680363,-79.289692,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,The Beaches,3,43.680493,-79.288846,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,The Beaches,6,43.672864,-79.286784,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,The Beaches,7,43.678879,-79.297734,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1


Drop the Latitudes and Longitude

In [8]:
Edibles_onehot.drop(['lat','lng','Unnamed: 0'],axis=1,inplace=True)

Group the data by neighborhoods and then create a Total column to sum the number of venues in each

In [9]:
Edibles_onehot_grouped = Edibles_onehot.groupby('Neighborhood').sum()
#Edibles_onehot_grouped = Edibles_onehot_grouped.apply(pd.to_numeric) # Convert to numeric values

#Add all values for restaurants in one column and other venues in another column
restaurtant_list =[]

All_feat = list(Edibles_onehot_grouped.columns)
for counter, feat in enumerate(All_feat):
    if feat.find('Restaurant') !=(-1):
        restaurtant_list.append(feat)
        
Edibles_onehot_grouped['Restaurant_Total'] = Edibles_onehot_grouped[restaurtant_list].sum(axis=1)
Edibles_onehot_grouped['Total'] = Edibles_onehot_grouped.sum(axis=1)
Edibles_onehot_grouped.drop(restaurtant_list, axis=1, inplace=True)
other_columns = ['Business reply mail Processing Centre, South Central Letter Processing Plant Toronto','India Bazaar, The Beaches West','Studio District','The Beaches']
Edibles_onehot_grouped.drop(other_columns, axis=1, inplace=True)
#Edibles_onehot_grouped.reset_index(inplace=True)
Edibles_onehot_grouped

,Bagels On Fire,Beacher Cafe,Beaches Bake Shop,COBS Bread,Cool Runnings,Delhi Bistro,Dyson's valu-mart,Foodland - Toronto,Garden Gate Resturant,Green Basil,Hero Certified Burgers,Honey's Beestro,Kyouka Ramen,Life Is Sweet,No Bull Burger,Souvlaki Hut,Subway,Sunset Grill,The Art of Cheese,The Beech Tree,The Big Carrot Natural Food Market,The Goof,The Real Jerk Beaches,The Salty Dog,Tori's Bakeshop,Veloute Bistro,Vi Vetha,Xola,Yumei Sushi,Bagel Shop,Bakery,Breakfast Spot,Burger Joint,Cheese Shop,Cupcake Shop,Diner,Gastropub,Grocery Store,Health Food Store,Sandwich Place,Restaurant_Total,Total
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,4,2,2,2,2,2,2,2,2,2,2,2,2,2,4,6,4,2,2,2,4,4,2,4,36,234
"India Bazaar, The Beaches West",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,2,1,1,1,2,2,1,2,18,117
Studio District,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,2,1,1,1,2,2,1,2,18,117
The Beaches,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,2,1,1,1,2,2,1,2,18,117


From the total column, we can see that three out of the four unique neighborhoods are the same in terms of number 
of venues. Hence, expect the algorithm to cluster the neighborhoods into two.

Let's pre-process the data to build the model

In [18]:
X = Edibles_onehot_grouped
#from sklearn import preprocessing
#X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

,Bagels On Fire,Beacher Cafe,Beaches Bake Shop,COBS Bread,Cool Runnings,Delhi Bistro,Dyson's valu-mart,Foodland - Toronto,Garden Gate Resturant,Green Basil,Hero Certified Burgers,Honey's Beestro,Kyouka Ramen,Life Is Sweet,No Bull Burger,Souvlaki Hut,Subway,Sunset Grill,The Art of Cheese,The Beech Tree,The Big Carrot Natural Food Market,The Goof,The Real Jerk Beaches,The Salty Dog,Tori's Bakeshop,Veloute Bistro,Vi Vetha,Xola,Yumei Sushi,Bagel Shop,Bakery,Breakfast Spot,Burger Joint,Cheese Shop,Cupcake Shop,Diner,Gastropub,Grocery Store,Health Food Store,Sandwich Place,Restaurant_Total,Total
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,4,2,2,2,2,2,2,2,2,2,2,2,2,2,4,6,4,2,2,2,4,4,2,4,36,234
"India Bazaar, The Beaches West",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,2,1,1,1,2,2,1,2,18,117
Studio District,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,2,1,1,1,2,2,1,2,18,117
The Beaches,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,2,1,1,1,2,2,1,2,18,117


In [18]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 4

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(X)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (4). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 1, 1, 1], dtype=int32)

The label returns only two unique clusters which is exactly as expected.

In [19]:
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = Edibles_onehot_grouped.columns
means_df.index = ['A1','A2','A3','A4']
means_df.sort_values(axis = 0, by = ['Total'], ascending=False) #Sort the values in Descending order to return the best neighborhood
means_df.head()

,Bagels On Fire,Beacher Cafe,Beaches Bake Shop,COBS Bread,Cool Runnings,Delhi Bistro,Dyson's valu-mart,Foodland - Toronto,Garden Gate Resturant,Green Basil,Hero Certified Burgers,Honey's Beestro,Kyouka Ramen,Life Is Sweet,No Bull Burger,Souvlaki Hut,Subway,Sunset Grill,The Art of Cheese,The Beech Tree,The Big Carrot Natural Food Market,The Goof,The Real Jerk Beaches,The Salty Dog,Tori's Bakeshop,Veloute Bistro,Vi Vetha,Xola,Yumei Sushi,Bagel Shop,Bakery,Breakfast Spot,Burger Joint,Cheese Shop,Cupcake Shop,Diner,Gastropub,Grocery Store,Health Food Store,Sandwich Place,Restaurant_Total,Total
A1,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,6.0,4.0,2.0,2.0,2.0,4.0,4.0,2.0,4.0,36.0,234.0
A2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,18.0,117.0
A3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,18.0,117.0
A4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,18.0,117.0


In [20]:
kneigh_summary = pd.DataFrame([Edibles_onehot_grouped.index, 1 + kmeans.labels_]).T
kneigh_summary.columns = ['Neighborhood', 'Group']
kneigh_summary

,Neighborhood,Group
0,"Business reply mail Processing Centre, South C...",1
1,"India Bazaar, The Beaches West",2
2,Studio District,2
3,The Beaches,2


Print the best neighborhood

In [21]:
kneigh_summary[kneigh_summary['Group'] == 1]

,Neighborhood,Group
0,"Business reply mail Processing Centre, South C...",1


In [24]:
top_neigh = list(kneigh_summary[kneigh_summary['Group'] == 1]['Neighborhood'])[0]
Toronto_data[Toronto_data['Neighborhood'] == top_neigh].iloc[0,1:5].to_dict()

{'Postal Code': 'M7Y',
 'Borough': 'East Toronto',
 'Neighborhood': 'Business reply mail Processing Centre, South Central Letter Processing Plant Toronto',
 'Latitude': 43.6627439}

In [27]:
#All remaining neighborhoods are clustered as the same
kneigh_summary[kneigh_summary['Group'] == 2]

,Neighborhood,Group
1,"India Bazaar, The Beaches West",2
2,Studio District,2
3,The Beaches,2


**Thank you for reading this notebook**